In [7]:
# Install the trueskill library if you haven't already
# !pip install trueskill

import trueskill
import random
import pandas as pd

# Initialize the TrueSkill environment
ts = trueskill.TrueSkill(draw_probability=0.1)  # Adjust draw probability as needed

# Define the number of teams and matches
NUM_TEAMS = 6
NUM_MATCHES = 1000

# Initialize teams with default ratings
teams = {}
for i in range(1, NUM_TEAMS + 1):
    team_name = f"Team_{i}"
    teams[team_name] = ts.Rating()

# Function to simulate a match between two teams
def simulate_match(ts_env, team1, team2):
    # Get the current ratings
    rating1 = teams[team1]
    rating2 = teams[team2]
    
    # Calculate each team's performance by sampling from their rating distributions
    perf1 = ts_env.expose(rating1)
    perf2 = ts_env.expose(rating2)
    
    # Determine the outcome
    if perf1 > perf2:
        # Team1 wins
        ranks = [0, 1]
    elif perf1 < perf2:
        # Team2 wins
        ranks = [1, 0]
    else:
        # Draw
        ranks = [0, 0]
    
    # Update the ratings
    new_ratings = ts_env.rate([ [rating1], [rating2] ], ranks)
    teams[team1] = new_ratings[0][0]
    teams[team2] = new_ratings[1][0]
    
    # Return the outcome
    return {
        'Team1': team1,
        'Team2': team2,
        'Team1_Rating': teams[team1],
        'Team2_Rating': teams[team2],
        'Winner': team1 if ranks[0] == 0 and ranks[1] == 1 else (team2 if ranks[0] == 1 and ranks[1] == 0 else 'Draw')
    }

# Simulate matches
match_results = []
for _ in range(NUM_MATCHES):
    # Randomly select two different teams
    team1, team2 = random.sample(list(teams.keys()), 2)
    result = simulate_match(ts, team1, team2)
    match_results.append(result)

# Create a DataFrame to display the match results
df_matches = pd.DataFrame(match_results)

# Calculate final ratings and confidence intervals
final_ratings = []
for team, rating in teams.items():
    # The rating object has a mean (mu) and standard deviation (sigma)
    # The conservative estimate of skill is called the "TrueSkill" score: mu - 3*sigma
    trueskill_score = rating.mu - 3 * rating.sigma
    final_ratings.append({
        'Team': team,
        'Rating_Mean': rating.mu,
        'Rating_StdDev': rating.sigma,
        'TrueSkill_Score': trueskill_score
    })

# Create a DataFrame to display the final team ratings
df_ratings = pd.DataFrame(final_ratings)
df_ratings.sort_values(by='TrueSkill_Score', ascending=False, inplace=True)

# Display the match results
print("Match Results:")
print(df_matches[['Team1', 'Team2', 'Winner']])

# Display the final team rankings
print("\nFinal Team Rankings:")
print(df_ratings[['Team', 'Rating_Mean', 'Rating_StdDev', 'TrueSkill_Score']])


Match Results:
      Team1   Team2  Winner
0    Team_1  Team_6    Draw
1    Team_5  Team_2    Draw
2    Team_5  Team_1    Draw
3    Team_2  Team_1  Team_1
4    Team_5  Team_2  Team_5
..      ...     ...     ...
995  Team_5  Team_2  Team_5
996  Team_2  Team_3  Team_2
997  Team_3  Team_6  Team_6
998  Team_1  Team_2  Team_1
999  Team_1  Team_6  Team_1

[1000 rows x 3 columns]

Final Team Rankings:
     Team  Rating_Mean  Rating_StdDev  TrueSkill_Score
4  Team_5    43.526834       1.537115        38.915490
0  Team_1    33.456835       1.205217        29.841183
5  Team_6    25.753140       1.154580        22.289400
1  Team_2    18.257247       1.177086        14.725988
3  Team_4    10.663600       1.248609         6.917774
2  Team_3    -0.500104       1.573035        -5.219208
